In [1]:
import os

if not os.path.exists('/kaggle/input/lyft-motion-prediction-autonomous-vehicles'):
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    !pip install -q kaggle
    
    from google.colab import files
    if not os.path.exists('kaggle.json'):
        files.upload()
        !mkdir ~/.kaggle
        !cp kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    #!kaggle datasets list
    !kaggle config path -p /content




    def kaggle_dataset_download_unzip(lfilename,dir):

        with zipfile.ZipFile(lfilename) as file: 
            file.extractall(dir)
        os.remove(lfilename)

    dir='/content/lyft-motion'
    if not os.path.exists(dir):
        !kaggle datasets download -d aikhmelnytskyy/liftfirst
        lfilename='/content/liftfirst.zip'
        kaggle_dataset_download_unzip(lfilename,dir+'/scenes')

        !kaggle datasets download -d aikhmelnytskyy/lyftsecond
        lfilename='/content/lyftsecond.zip'
        kaggle_dataset_download_unzip(lfilename,dir)

        !kaggle datasets download -d huanvo/lyft-pretrained-model-hv
        lfilename='/content/lyft-pretrained-model-hv.zip'
        kaggle_dataset_download_unzip(lfilename,'/content/lyft-pretrained-model-hv')
        
        data_path='/content/lyft-motion/'
        weight_path='/content/lyft-pretrained-model-hv/model_multi_update_lyft_public.pth'
        model_path='/content/drive/My Drive/Models/'#!!!!!!!!!!
else:
    data_path='/kaggle/input/lyft-motion-prediction-autonomous-vehicles'
    weight_path='/kaggle/input/lyft-pretrained-model-hv/model_multi_update_lyft_public.pth' 
    model_path='' 

Mounted at /content/drive


Saving kaggle.json to kaggle.json
usage: kaggle config [-h] {view,set,unset} ...
kaggle config: error: argument command: invalid choice: 'path' (choose from 'view', 'set', 'unset')
100% 15.9G/15.9G [03:16<00:00, 71.5MB/s]
100% 15.9G/15.9G [03:16<00:00, 87.1MB/s]
100% 2.33G/2.34G [00:15<00:00, 145MB/s]
100% 2.34G/2.34G [00:15<00:00, 166MB/s]
 93% 133M/143M [00:01<00:00, 97.8MB/s]
100% 143M/143M [00:01<00:00, 114MB/s] 


In [2]:
# this script transports l5kit and dependencies
!pip -q install pymap3d==2.1.0 
!pip -q install protobuf==3.12.2 
!pip -q install transforms3d 
!pip -q install zarr 
!pip -q install ptable

!pip -q install --no-dependencies l5kit

     |████████████████████████████████| 1.3MB 4.3MB/s 
     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 133kB 4.4MB/s 
     |████████████████████████████████| 5.8MB 6.7MB/s 
     |████████████████████████████████| 92kB 4.1MB/s 


In [3]:
import l5kit
print(l5kit.__version__)

1.1.0


In [4]:
# import packages
from google.colab import files
import numpy as np
import torch
import gc, os

from torch import nn, optim
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18, resnet34, resnet50
from torchvision.models.densenet import densenet121
from tqdm import tqdm
from typing import Dict

from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer

In [5]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
# from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as plt

import os
import random
import time
from IPython.display import display
from tqdm import tqdm_notebook
import gc, psutil

import warnings
warnings.filterwarnings("ignore")

In [6]:
# Memory measurement
def memory(verbose=True):
    mem = psutil.virtual_memory()
    gb = 1024*1024*1024
    if verbose:
        print('Physical memory:',
              '%.2f GB (used),'%((mem.total - mem.available) / gb),
              '%.2f GB (available)'%((mem.available) / gb), '/',
              '%.2f GB'%(mem.total / gb))
    return (mem.total - mem.available) / gb

def gc_memory(verbose=True):
    m = gc.collect()
    if verbose:
        print('GC:', m, end=' | ')
        memory()

memory();

Physical memory: 1.08 GB (used), 24.43 GB (available) / 25.51 GB


In [7]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed(42)

In [22]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': '/content/lyft-motion',
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "model_resnet34_wegihted_retrain_baded_on_235000",
        'lr': 5e-6,
        'weight_path': '/content/drive/MyDrive/dropout03/model_resnet34_wegihted_output_235000.pth',
        'train': True,
        'predict': False,
    },
    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5,
    },
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 8,
        'shuffle': True,
        'num_workers': 1,
    },    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 128,
        'shuffle': False,
        'num_workers': 4,
    },
    'train_params': {
        # 'steps': 100,
        # 'update_steps': 10,
        # 'checkpoint_steps': 50,
        'steps': 100000,
        'update_steps': 200,
        'checkpoint_steps': 20000,
    }
}

In [23]:
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager()

In [24]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

In [25]:
class LyftMultiModel(nn.Module):
    def __init__(self, cfg: Dict, num_modes=3):
        super().__init__()

        architecture = cfg["model_params"]["model_architecture"]
        backbone = eval(architecture)(pretrained=True, progress=True)
        self.backbone = backbone

        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        # This is 512 for resnet18 and resnet34
        # And it is 2048 for the other resnets        
        if architecture == "resnet50":
            backbone_out_features = 2048
        else:
            backbone_out_features = 512
        
        self.dropout = nn.Dropout(p=0.3)

        # X, Y coords for the future positions (output shape: batch_sizex50x2)
        self.future_len = cfg["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes

        self.logit = nn.Linear(4096, out_features=self.num_preds + num_modes)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)

        x = self.head(x)
        x = self.logit(x)

        # pred (batch_size)x(modes)x(time)x(2D coords)
        # confidences (batch_size)x(modes)
        bs, _ = x.shape
        pred, confidences = torch.split(x, self.num_preds, dim=1)
        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)
        confidences = torch.softmax(confidences, dim=1)
        return pred, confidences

In [26]:

def forward(data, model, device, criterion=pytorch_neg_multi_log_likelihood_batch, compute_loss=True):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    preds, confidences = model(inputs)
    # skip compute loss if we are doing prediction
    loss = criterion(targets, preds, confidences, target_availabilities) if compute_loss else 0
    return loss, preds, confidences

In [27]:
%%time
# Build rasterizer
rasterizer = build_rasterizer(cfg, dm)

CPU times: user 5.41 s, sys: 353 ms, total: 5.76 s
Wall time: 5.41 s


In [28]:
%%time
# Train dataset
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open(cached=False)  # to prevent run out of memory
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], 
                              batch_size=train_cfg["batch_size"], num_workers=train_cfg["num_workers"])
print(train_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
CPU times: user 6.38 s, sys: 3.43 s, total: 9.81 s
Wall time: 9.54 s


In [29]:
# samples_weight = pd.read_csv("/content/drive/MyDrive/baseline improvements/w_traindata.csv")
# samples_weight = samples_weight[["0"]].to_numpy()

In [30]:
%%time
if cfg["model_params"]["train"]:
    # Train dataset
    train_cfg = cfg["train_data_loader"]
    train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open(cached=False)  # to prevent run out of memory
    train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
    
    # samples_weight = torch.from_numpy(samples_weight)
    # sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))   
    # train_dataloader = DataLoader(train_dataset, shuffle=False, 
    #                               batch_size=train_cfg["batch_size"], num_workers=train_cfg["num_workers"],sampler=sampler)

    train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], 
                                  batch_size=train_cfg["batch_size"], num_workers=train_cfg["num_workers"])
    print(train_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
CPU times: user 6.17 s, sys: 1.12 s, total: 7.29 s
Wall time: 6.97 s


In [31]:
len(train_dataloader)

2812089

In [32]:
len(train_dataset)

22496709

In [33]:
%%time
# ==== INIT MODEL=================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = LyftMultiModel(cfg)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=cfg["model_params"]["lr"])

#load weight if there is a pretrained model
# epoch = 1
# WEIGHT_FILE = cfg["model_params"]["weight_path"]
# checkpoint = torch.load(WEIGHT_FILE, map_location=device)
# model.load_state_dict(checkpoint)

WEIGHT_FILE = cfg["model_params"]["weight_path"]
checkpoint = torch.load(WEIGHT_FILE, map_location=device)
epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

print(f'device {device}')

device cuda:0
CPU times: user 849 ms, sys: 333 ms, total: 1.18 s
Wall time: 3.72 s


In [34]:
epoch

235000

In [ ]:
%%time
if cfg["model_params"]["train"]:
    tr_it = iter(train_dataloader)
    n_steps = cfg["train_params"]["steps"]
    progress_bar = tqdm_notebook(range(epoch, 1 + epoch + cfg["train_params"]["steps"]), mininterval=10.)
    losses = []
    iterations = []
    metrics = []
    memorys = []
    times = []
    model_name = cfg["model_params"]["model_name"]
    update_steps = cfg['train_params']['update_steps']
    checkpoint_steps = cfg['train_params']['checkpoint_steps']
    t_start = time.time()
    torch.set_grad_enabled(True)
        
    for i in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
        model.train()   # somehow we need this is ever batch or it perform very bad (not sure why)
        loss, _, _ = forward(data, model, device)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_v = loss.item()
        losses.append(loss_v)
        
        if i % update_steps == 0:
            mean_losses = np.mean(losses)
            timespent = (time.time() - t_start) / 60
            print('i: %5d'%i,
                  'loss: %10.5f'%loss_v, 'loss(avg): %10.5f'%mean_losses, 
                  '%.2fmins'%timespent, end=' | ')
            mem = memory()
            # if i % checkpoint_steps == 0:
            #     torch.save(model.state_dict(), f'{model_name}_{i}.pth')
            #     torch.save(optimizer.state_dict(), f'{model_name}_optimizer_{i}.pth')
            if i % checkpoint_steps == 0:
                torch.save({'epoch': i,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict()},
                           f'/content/drive/My Drive/dropout03/{model_name}_output_{i}.pth') 
            iterations.append(i)
            metrics.append(mean_losses)
            memorys.append(mem)
            times.append(timespent)
    
    results = pd.DataFrame({
        'iterations': iterations, 
        'metrics (avg)': metrics,
        'elapsed_time (mins)': times,
        'memory (GB)': memorys,
    })
    results.to_csv(f'/content/drive/My Drive/dropout03/train_metrics_{model_name}_{i}.csv', index=False)
    print(f'Total training time is {(time.time() - t_start) / 60} mins')
    memory()
    display(results)


i: 235000 loss:   18.95519 loss(avg):   18.95519 0.03mins | Physical memory: 3.88 GB (used), 21.63 GB (available) / 25.51 GB
i: 235200 loss:    2.57253 loss(avg):   16.31812 2.24mins | Physical memory: 3.87 GB (used), 21.65 GB (available) / 25.51 GB
i: 235400 loss:   11.92922 loss(avg):   16.03517 4.35mins | Physical memory: 3.86 GB (used), 21.65 GB (available) / 25.51 GB
i: 235600 loss:   22.03712 loss(avg):   16.28435 6.44mins | Physical memory: 3.99 GB (used), 21.52 GB (available) / 25.51 GB
i: 235800 loss:   16.52003 loss(avg):   16.33897 8.45mins | Physical memory: 4.09 GB (used), 21.43 GB (available) / 25.51 GB
i: 236000 loss:    4.37050 loss(avg):   15.69369 10.51mins | Physical memory: 4.08 GB (used), 21.43 GB (available) / 25.51 GB
i: 236200 loss:    6.53674 loss(avg):   16.05157 12.50mins | Physical memory: 4.13 GB (used), 21.38 GB (available) / 25.51 GB
i: 236400 loss:   16.44347 loss(avg):   15.68606 14.52mins | Physical memory: 4.36 GB (used), 21.15 GB (available) / 25.51 

In [ ]:
if cfg["model_params"]["train"]:
    plt.figure(figsize=(12, 4))
    plt.plot(results['iterations'], results['metrics (avg)'])
    plt.xlabel('steps'); plt.ylabel('metrics (avg)')
    plt.grid(); plt.show()

    plt.figure(figsize=(12, 4))
    plt.plot(results['iterations'], results['memory (GB)'])
    plt.xlabel('steps'); plt.ylabel('memory (GB)')
    plt.grid(); plt.show()

    plt.figure(figsize=(12, 4))
    plt.plot(results['iterations'], results['elapsed_time (mins)'])
    plt.xlabel('steps'); plt.ylabel('elapsed_time (mins)')
    plt.grid(); plt.show()